In [24]:
import sys; sys.path.append('..')
import pyzx as zx
%config InlineBackend.figure_format = 'svg'

In [26]:
c = zx.qasm("""
qreg q[4];
cx q[0], q[1];
cx q[1],q[3];
t q[2];
""")
g = c.to_graph()
zx.draw(g)

In [27]:
c = zx.qasm("""
qreg q[2];
rx(0.5*pi) q[1];
t q[0];
cx q[0], q[1];
cx q[1], q[0];
cx q[0], q[1];
tdg q[1];
rx(-0.5*pi) q[0];
""")
zx.draw(c)

In [28]:
c.gates

[XPhase(1,phase=1/2),
 T(0),
 CNOT(0,1),
 CNOT(1,0),
 CNOT(0,1),
 T*(1),
 XPhase(0,phase=-1/2)]

In [29]:
g = c.to_graph()
g

Graph(14 vertices, 15 edges)

In [30]:
zx.draw(g)

In [31]:
zx.simplify.spider_simp(g)
zx.draw(g)

spider_simp: 1. 1.  2 iterations


In [34]:
zx.full_reduce(g)
zx.draw(g)

In [37]:
g = zx.sqasm("""
qreg S[1];
qreg q[3];
t S[0];
cx q[0], S[0];
cx q[1], S[0];
cx q[2], S[0];
""")
zx.draw(g)

In [39]:
g = zx.sqasm("""
qreg S[2];
qreg q[3];
t S[0];
cx q[0], S[0];
cx q[1], S[0];
cx q[2], S[0];
tdg S[1];
cx q[0], S[1];
cx q[1], S[1];
cx q[2], S[1];
""")
zx.draw(g)

In [40]:
zx.clifford_simp(g)
zx.draw(g)

In [41]:
zx.full_reduce(g)
zx.draw(g)

In [42]:
g = zx.Circuit.load("test.qsim").to_graph()
zx.draw(g)

In [43]:
g1 = g.copy()
g1.map_qubits([
    (0,0), (1, 0), (2, 0), (3, 0),
    (0,1), (1, 1), (2, 1), (3, 1),
    (0,2), (1, 2), (2, 2), (3, 2),
    (0,3), (1, 3), (2, 3), (3, 3)
])
zx.draw(g1)

In [44]:
zx.full_reduce(g1)
zx.draw(g1)

In [45]:
def t_optimiser(c):
    g = c.to_graph()
    g = zx.simplify.teleport_reduce(g)
    c_opt = zx.Circuit.from_graph(g).split_phase_gates().to_basic_gates()
    return zx.optimize.basic_optimization(c_opt).to_basic_gates()

In [46]:
c = zx.Circuit.load('../circuits/Fast/grover_5.qc')
zx.draw(c.to_graph())
print(zx.tcount(c))

336


In [47]:
c1 = t_optimiser(c)
zx.draw(c1.to_graph())
print(zx.tcount(c1))

166


In [48]:
c.verify_equality(c1)

True

In [49]:
c2 = c1.copy()
c2.add_circuit(c.adjoint())
g = c2.to_graph()
zx.simplify.full_reduce(g)
zx.draw(g)

In [50]:
c1.gates[10]

CNOT(0,6)

In [51]:
c1.gates[10] = zx.gates.T(6, adjoint=True)

In [52]:
c.verify_equality(c1)

False

In [53]:
c2 = c1.copy()
c2.add_circuit(c.adjoint())
g = c2.to_graph()
zx.simplify.full_reduce(g)
zx.draw(g)